In [15]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import geocoder

Code to Generate the Dataframe by scraping the wikipedia

In [20]:
results = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(results,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
data=My_table.findAll('td')
postalCode=[]
borough = []
neighborhood =[]
for i,link in enumerate(data):
    if i%3==0:
        postalCode.append(link.get_text())
    if i%3==1:
        borough.append(link.get_text())
    if i%3==2:
        neighborhood.append(link.get_text())
finalTable = {}
for i in range(len(postalCode)):
    if borough[i] != 'Not assigned':
        if postalCode[i] not in finalTable.keys():
            tempDict={}
            templist=[]
            if neighborhood[i].strip('\n') != 'Not assigned':
                templist.append(neighborhood[i].strip('\n'))
            else:
                templist.append(borough[i].strip('\n'))
            tempDict[borough[i].strip('\n')]=templist
            finalTable[postalCode[i]]=tempDict
        else:
            if neighborhood[i].strip('\n') != 'Not assigned':
                finalTable[postalCode[i]][borough[i].strip('\n')].append(neighborhood[i].strip('\n'))
            else:
                finalTable[postalCode[i]][borough[i].strip('\n')].append(borough[i].strip('\n'))

data={}
for k,v in finalTable.items():
    temp=[]
    temp.append(list(v.keys())[0])
    strtemp=""
    for i,j in enumerate(list(v.values())[0]):
        strtemp+=j
        if i != len(list(v.values())[0])-1:
            strtemp+=','
    temp.append(strtemp)
    data[k]=temp
df=pd.DataFrame.from_dict(data, orient='index')
df.reset_index(inplace=True)
df.rename(index=str, columns={'index': "PostalCode", 0: "Borough",1: "Neighborhood"},inplace=True)

In [21]:
df.head()

,PostalCode,Borough,Neighborhood
0,M6R,West Toronto,"Parkdale,Roncesvalles"
1,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
2,M1N,Scarborough,"Birch Cliff,Cliffside West"
3,M5G,Downtown Toronto,Central Bay Street
4,M3L,North York,Downsview West


In [4]:
def get_latlng(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
##Sample Test
get_latlng('M2J')

[43.780880000000025, -79.34779577599994]

Getting PostalCodes for all Boroughs and fetching Lattitude and longitude using above function 
and adding Columns and Data in original Dataframe

In [5]:
postal_codes = df['PostalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [6]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [29]:
df= df.assign(Latitude=list(df_coords['Latitude']))
df= df.assign(Longitude=list(df_coords['Longitude']))

In [31]:
df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M6R,West Toronto,"Parkdale,Roncesvalles",43.647870,-79.449762
1,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
2,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163
3,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.384930
4,M3L,North York,Downsview West,43.740885,-79.505027
5,M6B,North York,Glencairn,43.707990,-79.448367
6,M2K,North York,Bayview Village,43.781015,-79.380542
7,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653690,-79.511117
8,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.639410,-79.424362
9,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.609870,-79.498178
